# Notebook 8 - Représentation des connaissances

CSI4506 Intelligence Artificielle  
Automne 2020  
Préparé par Julian Templeton, Caroline Barrière et Joel Muteba

***INTRODUCTION***:  

Lors de la lecture de texte, comprendre le type d'entités avec le texte permet d'extraire des informations supplémentaires sur l'entité. Grâce à l'utilisation de la reconnaissance d'entités nommées (Named Entity Recognition, NER), nous sommes en mesure de déterminer si une entité est une personne, une organisation, un pays, ... Lors de l'exploration de texte en ligne, nous voyons aussi occasionnellement que les entités ont des liens cliquables vers des pages Web avec plus d'informations sur le entité. Il s'agit d'une forme d'amélioration du texte pour permettre aux lecteurs d'accéder facilement aux informations nécessaires pour comprendre chaque entité à partir du texte et de son contenu.

Dans ce notebook, nous revisiterons l'ensemble de données d'actualités liées à Covid-19 du notebook 7 pour explorer comment nous pouvons améliorer la désambiguïsation du NER de spaCy et améliorer le texte des articles de presse grâce à l'utilisation du linking d'entités. Cela se fera en deux parties, où nous utiliserons d'abord la cohérence du texte pour la désambiguïsation NER, puis nous effectuons une amélioration du texte avec le linking d'entités.

**Pour ce notebook, ne modifiez pas les définitions de fonction et assurez-vous d'utiliser la configuration qui vous est fournie ET ne soumettez que ce fichier, rien d'autre n'est nécessaire**.

Ce notebook utilise des bibliothèques qui ont été utilisées dans les notebooks précédents, notamment spaCy et pandas. Rappelez-vous que si vous rencontrez des problèmes avec le chargement de «en» pour commenter cette ligne et décommenter la ligne de code incluse (de la même manière que vous avez pu le faire dans le notebook 7).

***DEVOIR***:  

Parcourez le notebook en exécutant chaque cellule, une à la fois.
Recherchez **(TO DO)** pour les tâches que vous devez effectuer. Ne modifiez pas le code en dehors des questions auxquelles vous êtes invité à répondre à moins que cela ne vous soit spécifiquement demandé. Une fois que vous avez terminé, signez le notebook (à la fin du notebook) et soumettez-le.

*Le notebook sera noté le 30.
Chaque **(TO DO)** est associé à un certain nombre de points.*
***

In [1]:
# Before starting we will import every module that we will be using
import spacy
import pandas as pd

In [2]:
# The core spacy object that will be used for tokenization, lemmatization, POS Tagging, ...
# Note that this is specifically for the English language and requires the English package to be installed
# via pip to work as intended.

sp = spacy.load('en')

# If the above causes an error after installing the package described in (2), install the package described
# in the Note section within the introduction and run this line of code instead of the above.
#sp = spacy.load('en_core_web_sm')

**PARTIE 1 - Cohérence du texte pour la désambiguïsation des entités nommées**  
  
Pour la première partie de ce notebook, nous utiliserons les modules de * spaCy * pour aider à éliminer l'ambiguïté du NER et à améliorer les résultats avec la cohérence du texte sur les documents du fichier inclus sur les articles de presse liés à Covid-19 de CBC News (le même fichier du notebook 7). Nous commencerons par examiner la désambiguïsation du NER effectuée par spaCy et penserons à quelques méthodes simples pour utiliser la cohérence des entités dans le texte afin d'améliorer potentiellement la désambiguïsation du NER.


Comme pour le dernier notebook, l'ensemble de données est inclus avec ce notebook, mais les détails le concernant peuvent être trouvés [ici](https://www.kaggle.com/ryanxjhan/cbc-news-coronavirus-articles-march-26?select=news.csv). La première chose que nous allons faire, comme d'habitude, est de charger le fichier dans un dataframe pandas.

In [3]:
# Read the dataset, show top ten rows
df = pd.read_csv("news.csv")
df.head(10)

,Unnamed: 0,authors,title,publish_date,description,text,url
0,0,[],'More vital now:' Gay-straight alliances go vi...,2020-05-03 1:30,Lily Overacker and Laurell Pallot start each g...,Lily Overacker and Laurell Pallot start each g...,https://www.cbc.ca/news/canada/calgary/gay-str...
1,1,[],Scientists aim to 'see' invisible transmission...,2020-05-02 8:00,Some researchers aim to learn more about how t...,"This is an excerpt from Second Opinion, a week...",https://www.cbc.ca/news/technology/droplet-tra...
2,2,['The Canadian Press'],Coronavirus: What's happening in Canada and ar...,2020-05-02 11:28,Canada's chief public health officer struck an...,The latest: The lives behind the numbers: Wha...,https://www.cbc.ca/news/canada/coronavirus-cov...
3,3,[],"B.C. announces 26 new coronavirus cases, new c...",2020-05-02 18:45,B.C. provincial health officer Dr. Bonnie Henr...,B.C. provincial health officer Dr. Bonnie Henr...,https://www.cbc.ca/news/canada/british-columbi...
4,4,[],"B.C. announces 26 new coronavirus cases, new c...",2020-05-02 18:45,B.C. provincial health officer Dr. Bonnie Henr...,B.C. provincial health officer Dr. Bonnie Henr...,https://www.cbc.ca/news/canada/british-columbi...
5,5,"['Senior Writer', 'Chris Arsenault Is A Senior...",Brazil has the most confirmed COVID-19 cases i...,2020-05-02 8:00,"From describing coronavirus as a ""little flu,""...","With infection rates spiralling, some big city...",https://www.cbc.ca/news/world/brazil-has-the-m...
6,6,['Cbc News'],The latest on the coronavirus outbreak for May 1,2020-05-01 20:43,The latest on the coronavirus outbreak from CB...,Coronavirus Brief (CBC) Canada is officiall...,https://www.cbc.ca/news/the-latest-on-the-coro...
7,7,['Cbc News'],Coronavirus: What's happening in Canada and ar...,2020-05-01 11:51,Nova Scotia announced Friday it is immediately...,The latest: The lives behind the numbers: Wha...,https://www.cbc.ca/news/canada/coronavirus-cov...
8,8,"['Senior Writer', ""Adam Miller Is Senior Digit...",Did the WHO mishandle the global coronavirus p...,2020-04-30 8:00,The World Health Organization has come under f...,The World Health Organization has come under f...,https://www.cbc.ca/news/health/coronavirus-who...
9,9,['Thomson Reuters'],Armed people in Michigan's legislature protest...,2020-04-30 21:37,"Hundreds of protesters, some armed, gathered a...","Hundreds of protesters, some armed, gathered a...",https://www.cbc.ca/news/world/protesters-michi...


Dans le notebook précédent, lorsque nous avons exploré comment spaCy peut effectuer les différentes étapes de la pipeline TAL, nous avons vu qu'il était capable d'effectuer la reconnaissance d'entités nommées (NER). Vous trouverez ci-dessous le même exemple que nous avons vu dans le dernier notebook pour montrer comment nous pouvons accéder aux prédictions de type NER de spaCy pour les tokens dans un texte.

In [4]:
# Same example from notebook 7, recall that we loop through the iterator found in the .ents property of a parsed sentence
sentence_example = "Government guidelines in Canada recommend that people stay at least two metres away from others as part of physical distancing measures to curb the spread of COVID-19."
sentence_example_content = sp(sentence_example)
# Loop through all tokens that contain a NER type and print the token along with the corresponding NER type
for token in sentence_example_content.ents:
    print("\"" + token.text + "\" is a " + token.label_ )

"Canada" is a GPE
"at least two metres" is a QUANTITY


**(TO DO) Q1**  

Avant d'effectuer un NER avec cohérence de texte, vous explorerez d'abord comment spaCy effectue la désambiguïsation du NER. Dans le texte du ***second document*** (index 1) de notre corpus de documents, quels mots sont *PER* (spaCy utilise le type *PERSON*, plutôt que *PER*), *ORG* (Organisation) et *GPE* (entité géopolitique). Vous devez effectuer les opérations suivantes pour cette question:

a) Imprimez chaque *PER*, *ORG* et *GPE* avec son type NER depuis spaCy.

b) Toutes ces prédictions de type NER sont-elles correctes? Sinon, donnez trois exemples de sorties incorrectes.

c) Est-ce que certains des problèmes avec les prédictions de type NER proviennent d'une étape antérieure dans la pipeline TAL qui est effectuée par spaCy? Décrivez le problème pour deux exemples de la sortie ci-dessus.

**(TO DO) Q1 (a) - 2 points**  

Imprimez chaque *PER*, *ORG* et *GPE* avec son type NER depuis spaCy

In [5]:
# Select the second document (index 1)
doc = df["text"][1]
# TODO
doc_content = sp(doc)
for token in doc_content.ents:
    if token.label_ == "ORG" or token.label_ == "PER" or token.label_ == "GPE":
        print("\"" + token.text + "\" is a " + token.label_ )

"the World Health Organization" is a ORG
"Touches" is a ORG
"WHO" is a ORG
"the Public Health Agency" is a ORG
"Harvard School of Public Health" is a ORG
"Wells" is a ORG
"Canada" is a GPE
"the Fluid Dynamics of Disease Transmission Laboratory" is a ORG
"the Massachusetts Institute of Technology" is a ORG
"McMaster University" is a ORG
"RNA" is a GPE
"Wuhan" is a GPE
"China" is a GPE
"Nebraska" is a GPE
"Canada" is a GPE
"Sinai Health" is a ORG
"Toronto" is a GPE
"McGeer" is a ORG
"Bourouiba" is a GPE
"Bourouiba/MIT/" is a ORG
"Sunnybrook Hospital" is a ORG
"Toronto" is a GPE
"JAMA Insights" is a ORG
"N95" is a ORG
"U.S." is a GPE
"the New England Journal of Medicine" is a ORG
"the U.S. National Institutes of Health" is a ORG
"U.S. National Institutes of Health" is a ORG
"Journal of the Royal Society Interface" is a ORG
"U.S." is a GPE
"Singapore" is a GPE
"N95" is a ORG
"Penn State University/Journal of the Royal Society Interface" is a ORG
"The World Health Organization" is a ORG
"Lo

**(TO DO) Q1 (b) - 1 point**   
Toutes ces prédictions de type NER sont-elles correctes? Sinon, donnez deux exemples de sorties incorrectes.   

TODO ...

Non, quelques prédictions ne sont pas correctes. En effet, *Touches*, *Wells* et N95 ne sont pas des organisations. 


**(TO DO) Q1 (c) - 2 points**   
Est-ce que l'un des problèmes avec les prédictions de type NER provient d'une étape antérieure du pipeline NLP qui est effectuée par spaCy? Décrivez le problème pour deux exemples de la sortie ci-dessus. 

TODO ...   

Oui, il y a eu un problème à l'étape de tokenisation de la pipeline NLP. En effet, on peut remarquer que le mot *Bourouiba/MIT/* et *Penn State University/Journal of the Royal Society Interface* ont mal été séparés, ce qui a causé un effet boule de neige. Alors, ces mots ont été mal catégorisés comme étant des organisations par l'étape NER. Aussi, on peut remarque qu'il y a une répétition des mots, car *Bourouiba* et *Journal of the Royal Society Interface* ont déjà été catégorisés.

Maintenant que vous avez vu que spaCy NER ne fonctionne pas toujours correctement, nous allons essayer d'utiliser la cohérence du texte pour modifier les types de NER fournis par spaCy. En fait, spaCy affecte les types d'entités une phrase à la fois. Mais en regardant un document entier, et sachant que le texte est généralement cohérent, nous pouvons effectuer un post-traitement dans le module NER de spaCy et corriger certaines erreurs. Par texte cohérent, nous entendons, par exemple, que si une personne est mentionnée avec un nom particulier, par ex. *McGeer*, il y a de fortes chances que chaque fois que nous voyons *McGeer* dans le document, ce soit la même personne. Il est donc peu probable que *McGeer* soit une fois une personne et une fois une organisation. Ce n'est pas toujours vrai, mais c'est une hypothèse courante. Par conséquent, nous explorerons deux stratégies différentes pour utiliser la cohérence du texte pour post-traiter la sortie du module spaCy NER.

La première stratégie (*explorée en Q2 / Q3*) est de trouver, parmi tous les types de NER assignés, lequel est le plus fréquent. Par exemple, l'entité *Bourouiba* s'est vu attribuer 1 fois GPE et 3 fois PERSON, donc ces informations peuvent être utilisées pour modifier le type GPE et le changer en PERSON.

La deuxième stratégie (explorée à la Q4) est d'essayer de trouver une forme plus longue dans le texte. Puisque cette forme plus longue devrait être moins ambiguë, nous pouvons l'utiliser pour lever l'ambiguïté des formes plus courtes et plus ambiguës. Par exemple, *Lydia Bourouiba* apparaît dans le texte et se voit attribuer PERSONNE. Nous pouvons utiliser ces informations pour attribuer d'autres occurrences de la forme abrégée *Bourouiba* à également PERSONNE.

Une fois que nous avons défini ces deux stratégies, elles peuvent être combinées de différentes manières. Ainsi, à la Q5, il vous est demandé de combiner les deux stratégies dans un composant de post-traitement pour le module spaCy NER. Bien sûr, utiliser cette cohérence de texte ne fonctionnera pas à chaque fois, et introduira malheureusement quelques erreurs ... Mais essayons.

Dans le reste de cette section, nous travaillerons avec le septième document du corpus (index 6). Ci-dessous, nous chargeons le document et explorons toutes les entités dans le document avec leur type NER correspondant.

In [6]:
# Load the document's text for the seventh document (index 6)
doc = df["text"][6]
# Parse the text with spaCy
doc_sp = sp(doc)

In [7]:
# Display all entities from the text along with their index in the .ents iterator and the
# corresponding NER type
for i, token in enumerate(doc_sp.ents):
    print(str(i) + ": \"" + token.text + "\" is a " + token.label_ )

0: "Coronavirus Brief" is a ORG
1: "CBC" is a ORG
2: "Canada" is a GPE
3: "C.D. Howe" is a PERSON
4: "Ontario" is a GPE
5: "Monday" is a DATE
6: "Alberta" is a GPE
7: "first" is a ORDINAL
8: "Saturday" is a DATE
9: "Air Canada" is a ORG
10: "Christmas" is a DATE
11: "Canadians" is a NORP
12: "more than $1.2 million" is a MONEY
13: "England" is a GPE
14: "Peter Cziborra/Reuters" is a PERSON
15: "months" is a DATE
16: "CBC" is a ORG
17: "Andre Mayer" is a PERSON
18: "Canada" is a GPE
19: "19th-century" is a DATE
20: "2013" is a DATE
21: "Calgary" is a GPE
22: "John Brown" is a PERSON
23: "the University of Calgary" is a ORG
24: "two-metre" is a TIME
25: "Last week" is a DATE
26: "Italian" is a NORP
27: "Milan" is a GPE
28: "35 kilometres" is a QUANTITY
29: "Berlin" is a GPE
30: "Budapest" is a GPE
31: "Mexico City" is a GPE
32: "Ahsan Habib" is a PERSON
33: "Dalhousie University" is a ORG
34: "U.S." is a GPE
35: "Atlanta" is a GPE
36: "Chicago" is a GPE
37: "Denver" is a GPE
38: "Habib" 

**(TO DO) Q2 - 3 points**  
Comme vous pouvez le voir dans les résultats, parfois la même entité s'est vu attribuer différents types d'entités (par exemple, dans le document pour Q1 *McGeer* était une fois ORG, une fois PERSON) puisque l'algorithme NER regarde phrase par phrase. Dans la fonction suivante, le but sera de trouver tous les types d'entités possibles affectés à une seule entité.

Complétez la définition de la fonction *find_entity_types* ci-dessous. Cette fonction accepte en entrée une entité spaCy spécifique définie par le paramètre *entity* (à partir de l'itérable *.ents* des entités) et une liste de toutes les entités spaCy définies par le paramètre *entity*.

La fonction doit trouver toutes les entités du même nom que *entité* à partir des *entités* (la même forme de surface). Pour chaque correspondance entre les entités, ajoutez le type NER de l'entité de la liste au dictionnaire *type_counts* et suivez le nombre de fois où chaque type NER apparaît.

Ex: type_counts \[NER type\] = nombre total de fois où le décompte apparaît

In [12]:
def find_entity_types(entity, entities):
    '''
    Given a specific entity and a list of entities, finds all entities from the list that match the specified
    entity, but are of a different type.
    
    Returns the different NER types that have been classified for an entity and the count per NER type
    as a dictionary with the keys as the NER type and the value as the count
    '''
    type_counts = { }
    for token in entities:
        if str(token.text) == str(entity): #if the token is equal to the entity 
            if token.label_ in type_counts: #if the token label is in the dictionnary, update the value
                count = type_counts[token.label_]
                type_counts[token.label_] = (count + 1)
                
            else: #if not, put the token label with the value 1 
                type_counts[token.label_] = 1
                
    return type_counts

In [13]:
# Test the above to find the result when checking for the types of the entity 'Kenney' 
# from the document loaded above
print("All possible NER types for \"" + doc_sp.ents[85].text + "\" are " + str(find_entity_types(doc_sp.ents[85], doc_sp.ents)))

All possible NER types for "Kenney" are {'ORG': 1, 'PERSON': 1}


**(TO DO) Q3 - 2 points**  

Dans la méthode précédente, *find_entity_types*, nous avons trouvé tous les types d'entités possibles pour une seule entité. Maintenant, nous voulons les utiliser pour trouver le type le plus courant. Si nous regardons à nouveau les résultats pour Q1, dans le cas de *McGeer*, c'est une égalité. Mais pour *Bourouiba*, il existe un type GPE et 3 types PERSON, donc le plus courant serait PERSON.

Complétez la définition de la fonction *most_common_type* ci-dessous. Cette fonction accepte en entrée une entité spaCy spécifique définie par le paramètre *entity* (à partir de l'itérable *.ents* des entités) et une liste de toutes les entités spaCy définies par le paramètre *entity*.

Remarque: vous pouvez régler les cas d'égalité à votre guise.

In [24]:
def most_common_type(entity, entities):
    '''
    Given a specific entity and a list of entities, find the most similar entities and assign the
    NER type to entity based on the most common NER type assigned to entities of the same name (if there
    is a tie, you decide how to handle this).
    
    Returns the most common NER type based on similar entities
    '''
    # TODO
    entity_types = find_entity_types(entity, entities) #find the entity types of an entity
    max_value = max(entity_types.values()) #find the max value of the dictionnary
    common_NER = ""
    i = 0
    for NER, count in entity_types.items(): #loop throught dictionnary 
        if count == max_value: #if an item is equal to the max value, append the NER to the string
            if(i > 0):
                common_NER += " and " + NER
            else: 
                common_NER += NER
            
        i += 1 #counter if there is more than one common NER
            
    return common_NER
    

In [25]:
# Test the above to find the result when checking for the types of the entity 'Kenney' 
# from the document loaded above
print("The most common NER type to \"" + doc_sp.ents[85].text + "\" is " + most_common_type(doc_sp.ents[85], doc_sp.ents))

The most common NER type to "Kenney" is ORG and PERSON


**(TO DO) Q4 - 2 points**  

Nous allons maintenant travailler avec une méthode légèrement plus sophistiquée. Nous travaillerons à nouveau avec les mêmes *entité* et paramètres *entités*, mais cette fois vous devrez attribuer à *entité* le type NER d'une autre entité dans l'itérateur *entities*.

Plus précisément, vous devez parcourir les *entités* pour trouver une forme normalisée de *entité*. Dans ce scénario, toute entité contenant *entité* en tant que sous-chaîne sera considérée comme une sélection valide pour la forme normalisée (où l'entité sélectionnée n'a pas le même nom que *entité*). Si une forme normalisée est trouvée, retournez le type NER de cette entité, le nom de cette entité et l'entité elle-même.

Ex: *CBC News Network* est la forme normalisée de *CBC*. Ainsi, si cette entité est trouvée, retournez le type NER de l'entité (*ORG*) et le nom de l'entité (*CBC News Network*).

In [54]:
def assign_normalized_form(entity, entities):
    '''
    Given an entity and a list of entities, search the list of entities for any token that
    is does not have the exact same text as entity and assign entity that token's NER type
    if entity is a substring of that token.
    
    Returns the empty string if no normalized forms are found and the NER type of the normalized form if it is found.
    Also returns the name of the entity found, if any (along with the entity).
    '''
    # MAY BE DONE SO THAT THE LAST GETS ADDED INSTEAD, THIS IS FINE.
    # Recall to return the three requested components (NER type, the text, and the actual entity)
    normalized_form = ""
    for token in entities:
        if (str(entity) in str(token.text)) and (str(entity) != str(token.text)): #if the entity is in the token and they are not equal to each other, then the token is the normalized form of the entity
            normalized_form = str(entity) + " fait référence à " + token.text + " et est un " + token.label_ #write the correct sentence
    return normalized_form
    

In [55]:
# Test the above to find the result when checking for the types of the entity 'Kenney' 
# from the document loaded above
print(assign_normalized_form(doc_sp.ents[85], doc_sp.ents))
# Test the above to find the result when checking for the types of the entity 'CBC News' 
# from the document loaded above
print(assign_normalized_form(doc_sp.ents[153], doc_sp.ents))

Kenney fait référence à Jason Kenney et est un PERSON
CBC News fait référence à CBC News Network et est un ORG


**(TO DO) Q5**  

Maintenant que vous avez défini plusieurs algorithmes pour effectuer une désambiguïsation NER avec cohérence de texte, vous allez tester vos algorithmes et les utiliser pour définir une méthode légèrement plus robuste de désambiguïsation NER en combinant les techniques effectuées. Vous explorerez ensuite si ces techniques aident toujours à désambiguïser les NER.

a) Revisitez le document qui a été utilisé à Q1 (index 1) et, pour chaque entité, récupérez la forme normalisée de l'entité (le cas échéant) et n'affichez que les formes normalisées avec leurs types NER dans le format suivant (seulement s'il y a est une forme normalisée renvoyée):

&emsp; *Original_entity fait référence à Normalized_entity, et est un NER_Type_of_Normalized_Form*

b) Définissez un algorithme plus robuste qui combine les algorithmes conçus dans les dernières questions. Cet algorithme doit accepter une entité spécifique et une liste d'entités comme entrée, rechercher la forme normalisée de l'entité spécifique (le cas échéant) et renvoyer un type NER pour la forme normalisée basée sur le type NER le plus courant pour cette entité. Si aucune forme normalisée n'est trouvée, l'algorithme doit continuer en utilisant l'entité spécifique. Vous devez également renvoyer le nom de la forme normalisée (ou de l'entité d'origine s'il n'y a pas de forme normalisée).

c) Pour le septième document (index 6), exécutez l'algorithme défini en b) pour chaque entité, en imprimant ce qui suit pour chaque entité:

&emsp; *Original_entity fait référence à Normalized_entity (si aucun, identique à l'original), et est un Most_common_NER_type_of_normalized_form*

d) Est-ce que l'un des résultats obtenus en effectuant une désambiguïsation NER avec la cohérence du texte Q5 (c) semble problématique? Donnez un exemple de problème qui se produit avec nos approches et expliquez pourquoi ce problème se produit.

**(TO DO) Q5 (a) - 1 point**     

a) Revisitez le document qui a été utilisé à Q1 (index 1) et, pour chaque entité, récupérez la forme normalisée de l'entité (le cas échéant) et n'affichez que les formes normalisées ainsi que leurs types NER dans le format suivant (seulement s'il y a est une forme normalisée renvoyée):

&emsp; *Original_entity fait référence à Normalized_entity, et est un NER_Type_of_Normalized_Form*

Par exemple "Bourouiba fait référence à Lydia Bourouiba, et est une PERSONNE" serait imprimé pour une entité.

In [56]:
# Select document 2
doc = df["text"][1]
sp_doc_test = sp(doc)

In [57]:
# TODO: Loop through and print the assigned phrase with the appropriate text
# Example of the print statement structure (from document 1): Bourouiba refers to Lydia Bourouiba, and is a PERSON

for entity in sp_doc_test.ents:
    if assign_normalized_form(entity, sp_doc_test.ents) != "":
        print(assign_normalized_form(entity, sp_doc_test.ents))

two metres fait référence à farther than two metres et est un QUANTITY
two fait référence à two-metre et est un QUANTITY
two metres fait référence à farther than two metres et est un QUANTITY
Wells fait référence à W.F. Wells et est un PERSON
Bourouiba fait référence à Bourouiba/MIT/ et est un ORG
Loeb fait référence à Mark Loeb et est un PERSON
Loeb fait référence à Mark Loeb et est un PERSON
McGeer fait référence à Allison McGeer et est un PERSON
McGeer fait référence à Allison McGeer et est un PERSON
2 fait référence à 2009 et est un DATE
Bourouiba fait référence à Bourouiba/MIT/ et est un ORG
Bourouiba fait référence à Bourouiba/MIT/ et est un ORG
Bourouiba fait référence à Bourouiba/MIT/ et est un ORG
two metres fait référence à farther than two metres et est un QUANTITY
two fait référence à two-metre et est un QUANTITY
two fait référence à two-metre et est un QUANTITY
Second fait référence à Second Opinion et est un LAW
McMaster fait référence à McMaster University et est un ORG


**(TO DO) Q5 (b) - 2 points**     

b) Définissez un algorithme plus robuste qui combine les algorithmes conçus dans les dernières questions. Cet algorithme doit accepter une entité spécifique et une liste d'entités en entrée, rechercher la forme normalisée de l'entité spécifique (le cas échéant) et renvoyer un type NER pour la forme normalisée basée sur le type NER le plus courant pour cette entité. Si aucune forme normalisée n'est trouvée, l'algorithme doit continuer en utilisant l'entité spécifique. Vous devez également renvoyer le nom de la forme normalisée (ou de l'entité d'origine s'il n'y a pas de forme normalisée).

In [61]:
def normalized_most_common_type(entity, entities):
    '''
    Determine the normalized form of an entity (if any; if none just use the entity) and
    return the most frequent NER type for that normalized form from a list of entities.
    '''
    # TODO (Recall to return the name and the NER type that is found)
    common_NER = ""
    normalized_most_common_type = ""
    for token in entities:
        if (str(entity) in str(token.text)) and (str(entity) != str(token.text)): #if the entity is in the token and they are not equal to each other, then the token is the normalized form of the entity
            common_NER = most_common_type(token.text, entities) #find the common NER of the token and assign it to the entity
            normalized_most_common_type = str(entity) + " fait référence à " + token.text + " et est un " + common_NER
    if normalized_most_common_type == "": #if there is no normalized form, then write the correct sentence
        common_NER = most_common_type(entity, entities)
        normalized_most_common_type = str(entity) + " fait référence à " + str(entity) + " et est un " + common_NER
    return normalized_most_common_type
        

**(TO DO) Q5 (c) - 1 point**   

c) Pour le septième document (index 6), exécutez l'algorithme défini en b) pour chaque entité, en imprimant ce qui suit pour chaque entité:

&emsp; *Original_entity fait référence à Normalized_entity (si aucun, identique à l'original), et est un Most_common_NER_type_of_normalized_form* 

In [62]:
# Load the document's text
doc = df["text"][6]
sp_doc_test = sp(doc)

In [63]:
# TODO: Loop through and print the assigned phrase with the appropriate text
for entity in sp_doc_test.ents:
    print(normalized_most_common_type(entity, sp_doc_test.ents))

Coronavirus Brief fait référence à Coronavirus Brief et est un ORG
CBC fait référence à CBC News et est un ORG
Canada fait référence à Health Canada et est un ORG
C.D. Howe fait référence à the C.D. Howe Institute's Business Cycle Council et est un ORG
Ontario fait référence à Ontario et est un GPE
Monday fait référence à Monday et est un DATE
Alberta fait référence à Alberta et est un GPE
first fait référence à first et est un ORDINAL
Saturday fait référence à Saturday et est un DATE
Air Canada fait référence à Air Canada et est un ORG
Christmas fait référence à Christmas et est un DATE
Canadians fait référence à Canadians et est un NORP
more than $1.2 million fait référence à more than $1.2 million et est un MONEY
England fait référence à England et est un GPE
Peter Cziborra/Reuters fait référence à Peter Cziborra/Reuters et est un PERSON
months fait référence à less than two months old et est un DATE
CBC fait référence à CBC News et est un ORG
Andre Mayer fait référence à Andre Maye

**(TO DO) Q5 (d) - 2 points**     

d) Est-ce que l'un des résultats obtenus en effectuant une désambiguïsation NER avec la cohérence du texte Q5 (c) semble problématique? Donnez un exemple de problème qui se produit avec nos approches et expliquez pourquoi ce problème se produit.

TODO ... 

Oui, l'un des résulats obtenus semble problématique avec la cohérence du texte. En effet, l'énoncé *Canada fait référence à Health Canada et est un ORG* est fausse. Ce problème se produit, car lorsqu'on cherche la forme normalisée d'un mot, on cherche si ce mot est présent dans autre mot. Dans cet exemple, *Health Canada* est bien une organisation, mais Canada ne l'es pas. Ainsi, lorsqu'on cherche la forme normalisée et qu'on imprime le NER le plus commun, on ne tient pas en compte le sens d'un mot. 


**PARTIE 2 - Linking d'entité / Amélioration du texte**  

Pour la deuxième partie de ce notebook, nous explorerons comment nous pouvons améliorer le texte des documents. Dans ce scénario, nous améliorerons le texte en effectuant un linking d'entité. Cela signifie que nous allons essayer plusieurs méthodes pour relier les entités qui sont détectées par le NER de spaCy à une page Web active sur laquelle un lecteur peut cliquer pour obtenir plus d'informations sur l'entité. De nombreux sites Web, tels que Wikipedia, effectuent des linkings d'entités pour permettre d'obtenir plus de contexte lors de la lecture d'un document.

Avant d'aller directement dans un exemple à travers le code, voici un exemple de la façon dont un texte sans linking d'entité se compare à un texte avec linking d'entité:

Aucun linking d'entité:
Pendant la pandémie, des villes américaines telles qu'Atlanta, Chicago et Denver ont apporté plusieurs ajustements à leurs systèmes de transport en commun.

Avec linking d'entité:
Pendant la pandémie, des villes américaines telles que <a href="http://en.wikipedia.org/wiki/Atlanta"> Atlanta </a>, <a href = "http://en.wikipedia.org/wiki / Chicago "> Chicago </a> et <a href="http://en.wikipedia.org/wiki/Denver"> Denver </a> ont apporté plusieurs ajustements à leurs systèmes de transport en commun.

Étant donné que vous allez concevoir plusieurs méthodes pour effectuer un linking d'entité simple, voici un exemple qui montre comment nous pouvons effectuer manuellement la linking d'entités sans aucune ressource. Cela montrera comment cela peut être effectué afin que vous puissiez utiliser et créer des ressources pour créer vous-même des algorithmes de linking d'entités simples.

In [67]:
sentence_example = "During the pandemic, U.S. cities such as Atlanta, Chicago and Denver have made several adjustments to their transit systems"
# Parse the example sentence
text_sp = sp(sentence_example)
# This will store the enhanced version of the text
enhanced_text = sentence_example
# Loop through the entities that spaCy has found and replace them as needed to be in expanded form 
for token in text_sp.ents:
    if token.text == "Atlanta":
        enhanced_text = enhanced_text.replace(token.text, "<a href=\"http://en.wikipedia.org/wiki/Atlanta\">Atlanta</a>")
    elif token.text == "Chicago":
        enhanced_text = enhanced_text.replace(token.text, "<a href=\"http://en.wikipedia.org/wiki/Chicago\">Chicago</a>")
    elif token.text == "Denver":
        enhanced_text = enhanced_text.replace(token.text, "<a href=\"http://en.wikipedia.org/wiki/Denver\">Denver</a>")
    
# Write the result as an HTML file (open to view the enhanced text!)
with open("enhanced_example.html", "w", encoding="utf-8") as f:
    f.write(enhanced_text)
    f.close()

En ouvrant le fichier *Enhanced_example.html* qui se trouve maintenant dans le même répertoire que ce notebook, vous pourrez voir comment nous avons liés les entités du texte.

Cela dit, le processus ci-dessus est assez pauvre. Il fallait indiquer manuellement les entités avec lesquelles travailler et l'URL pour y établir un lien. Ainsi, vous répondrez aux questions pour le reste de cette section où vous utilisez et/ou créez des ressources qui ont été assemblées manuellement pour lier des entités dans des méthodes plus générales/robustes. Il existe de nombreuses techniques de correspondance de chaînes différentes qui peuvent être utilisées pour aider à le linking d'entités, mais nous nous en tiendrons aux approches de base pour ce notebook.

Dans la question suivante, vous commencerez à travailler avec des ressources externes. Ainsi, ci-dessous, nous chargeons le fichier *US_Cities.csv* à utiliser pour améliorer le texte avec les villes américaines dans la question suivante. Notez que chaque fichier contient deux colonnes; *Texte* et *URL*. *Texte* fait référence à un nom d'entité et *URL* fait référence à une *URL* correspondante qui fournit plus d'informations concernant le *Texte*. L'exemple ci-dessous montre comment ces fichiers doivent être chargés et sont accessibles.

In [68]:
# Start with the string match approach (exact match)
# File content extracted from https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population
df_cities = pd.read_csv("US_Cities.csv")
# Print the Text and URL from each row, showcasing how to loop through the contents 
for i, row in df_cities.iterrows():
    print(row["Text"] + " - " + row["URL"])

New York City - https://en.wikipedia.org/wiki/New_York_City
Los Angelas - https://en.wikipedia.org/wiki/Los_Angeles
Chicago - https://en.wikipedia.org/wiki/Chicago
Houston - https://en.wikipedia.org/wiki/Houston
Phoenix - https://en.wikipedia.org/wiki/Phoenix,_Arizona
Philadelphia - https://en.wikipedia.org/wiki/Philadelphia
San Antonio - https://en.wikipedia.org/wiki/San_Antonio
San Diego - https://en.wikipedia.org/wiki/San_Diego
Dallas - https://en.wikipedia.org/wiki/Dallas
San Jose - https://en.wikipedia.org/wiki/San_Jose,_California
Austin - https://en.wikipedia.org/wiki/Austin,_Texas
Jacksonville - https://en.wikipedia.org/wiki/Jacksonville,_Florida
Fort Worth - https://en.wikipedia.org/wiki/Fort_Worth,_Texas
Columbus - https://en.wikipedia.org/wiki/Columbus,_Ohio
Charlotte - https://en.wikipedia.org/wiki/Charlotte,_North_Carolina
San Francisco - https://en.wikipedia.org/wiki/San_Francisco
Indianapolis - https://en.wikipedia.org/wiki/Indianapolis
Seattle - https://en.wikipedia.org

**(TO DO) Q6 - 3 points**  

Complétez la fonction *enhan_text_with_resource* ci-dessous. Il reçoit le texte du document via *document_text*, le dataframe de la ressource externe pour améliorer le texte avec comme *resource_df*, et le nom du fichier dans lequel vous allez sortir les résultats (un fichier .html) comme *filename*.

Cette fonction analyse le texte du document et remplace toutes les *entités* (.ents) trouvées dans le texte par:
<a href=\"Some URL">Texte d'entité</a\>

Après avoir amélioré le texte avec le linking d'entité, nous écrivons le texte amélioré dans un fichier html et renvoyons le texte amélioré

In [97]:
def enhance_text_with_resource(document_text, resource_df, filename):
    '''
    With a resource and document's text, enhance any entity found in the resource by linking the entity to
    the appropriate webpage.
    Write the file to the appropriate filename and return the enhanced text
    '''
    enhanced_text = document_text
    # TODO: Parse the document with spaCy
    document_sp = sp(enhanced_text)
    # TODO: Go through the entities and edit the document's text accordingly
    # Note: Be sure to not duplicate your enhancementes
    
    df_ressource = pd.read_csv(resource_df) #read csv file
    df_ressource.set_index("Text", inplace=True) #set the DataFrame index using existing columns 
    for token in document_sp.ents: 
        for index in df_ressource.index:
            if index in token.text: #if a word in the csv file and a token in a file match, enhance the text
                enhanced_text = enhanced_text.replace(token.text, ("<a href=\"" + str(df_ressource.loc[index]["URL"]) + "\">" + token.text + "</a>"))
    
    # Write the result as an HTML file
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(enhanced_text)
        f.close()
    return enhanced_text

**(TO DO) Q7 - 3 points**  

Grâce à l'algorithme d'amélioration du texte conçu (*enhan_text_with_resource*), vous allez maintenant tester la fonctionnalité lors de l'exécution de l'algorithme avec trois ressources différentes. Vous testerez l'algorithme pour chaque document déjà chargé dans la cellule de code et exécuterez les algorithmes avec les trois ressources suivantes:

1) Un fichier contenant plusieurs villes américaines: *US_Cities.csv*

2) Un fichier contenant toutes les provinces du Canada: *Canada_Provinces.csv*

3) Un fichier contenant plusieurs universités canadiennes: *Canada_Universities.csv*

In [102]:
# Enhance the text for the document below with the US cities
doc = df["text"][6]
# TODO ...
enhance_text_with_resource(doc, "US_Cities.csv", "US_Cities.html")

# Enhance the text for the document below with the Canadian provinces
# File extracted from https://en.wikipedia.org/wiki/Provinces_and_territories_of_Canada
doc = df["text"][53]
# TODO ...
enhance_text_with_resource(doc, "Canada_Provinces.csv", "Canada_Provinces.html")

# Enhance the text for the document below with the Canadian universities
# File extracted from https://en.wikipedia.org/wiki/List_of_universities_in_Canada
doc = df["text"][53]
# TODO ...
enhance_text_with_resource(doc, "Canada_Universities.csv", "Canada_Universities.html")

'The latest:  Federal government announces new measures to mobilize scientists. Saskatchewan unveils five-stage plan to reopen\xa0its\xa0economy. The Calgary Stampede is cancelled for the first time in nearly a century. North Korea claims it has had no coronavirus cases. COVID-19 has brought research into other medical conditions to a grinding halt. Ending Quebec\'s lockdown could mean freedom for some, but not for all. The pandemic is giving some countries political cover for controversial acts. INTERACTIVE |\xa0See the latest data on coronavirus cases\xa0in Canada. Have a coronavirus question or news tip for CBC News? Email: Covid@cbc.ca  The federal government has announced a $1.1-billion plan\xa0to marshal Canada\'s scientific community in the fight against the coronavirus, as some provinces with relatively fewer cases begin to weigh how they will relax restrictions put in place to slow the spread of\xa0COVID-19. Scientists around the globe are scrambling to come up with tests, tre

Maintenant, si vous ouvrez les fichiers HTML enregistrés, vous devriez constater que les mots qui apparaissent dans le texte et la ressource sont désormais directement liés aux informations pertinentes pour cette entité. Nous serions également en mesure d'améliorer un document avec de nombreuses ressources pour relier autant d'entités que possible.

**(TO DO) Q8 - 2 points**  
Parcourez les textes améliorés générés par vos tests dans Q7. Voyez-vous des universités qui ne sont pas liées lors de l'utilisation de la ressource universitaire? Pourquoi? Utilisez la cellule de code ci-dessous pour afficher tout ce que vous pourriez avoir besoin d'étudier (si vous n'avez rien remarqué des sorties précédentes) et répondez à la question dans la démarque sous cette cellule de code.

Remarque: pour le savoir, vous devez parcourir les fichiers .csv et le texte lui-même (à la fois le texte initial et la détection d'entité de spaCy).

In [103]:
# Look through any outputs that may seem off to help understand why (if not already known)
df_ressource = pd.read_csv("Canada_Universities.csv")
df_ressource.set_index("Text", inplace=True) #set the DataFrame index using existing columns 
print(df_ressource) #print the csv file

enhance_text_with_resource(doc, "Canada_Universities.csv", "Canada_Universities.html") #print the html file


                                                                              URL
Text                                                                             
University of Calgary           https://en.wikipedia.org/wiki/University_of_Al...
University of Calgary           https://en.wikipedia.org/wiki/University_of_Al...
Simon Fraser University         https://en.wikipedia.org/wiki/Simon_Fraser_Uni...
University of British Columbia  https://en.wikipedia.org/wiki/University_of_Br...
University of Manitoba          https://en.wikipedia.org/wiki/University_of_Ma...
University of Winnipeg          https://en.wikipedia.org/wiki/University_of_Wi...
Dalhousie University            https://en.wikipedia.org/wiki/Dalhousie_Univer...
Carleton University             https://en.wikipedia.org/wiki/Carleton_University
University of Ottawa            https://en.wikipedia.org/wiki/University_of_Ot...
University of Toronto           https://en.wikipedia.org/wiki/University_of_To...
York University 

'The latest:  Federal government announces new measures to mobilize scientists. Saskatchewan unveils five-stage plan to reopen\xa0its\xa0economy. The Calgary Stampede is cancelled for the first time in nearly a century. North Korea claims it has had no coronavirus cases. COVID-19 has brought research into other medical conditions to a grinding halt. Ending Quebec\'s lockdown could mean freedom for some, but not for all. The pandemic is giving some countries political cover for controversial acts. INTERACTIVE |\xa0See the latest data on coronavirus cases\xa0in Canada. Have a coronavirus question or news tip for CBC News? Email: Covid@cbc.ca  The federal government has announced a $1.1-billion plan\xa0to marshal Canada\'s scientific community in the fight against the coronavirus, as some provinces with relatively fewer cases begin to weigh how they will relax restrictions put in place to slow the spread of\xa0COVID-19. Scientists around the globe are scrambling to come up with tests, tre

TODO ... 

Comme on peut le remaquer ci-dessus, il y a des universités qui ne sont pas liées lors de l'utilisation de la ressource universitaire. En effet, on peut remarquer que l'université de Saskatchewan n'est pas présente dans le fichier csv, donc l'université ne va pas être liées à une page web dans le fichier html.

**(TO DO) Q9 - 2 points**  
Nous allons maintenant combiner une partie du travail effectué dans la partie 1 de ce notebook avec le travail effectué dans cette partie du notebook. Plus précisément, nous effectuerons une validation de type NER pour nous assurer que lorsque nous améliorons du texte avec une ressource, elle n'améliorera que les entités du type NER correct. Par exemple, lorsque nous utilisons la ressource des villes ou des provinces, assurez-vous que l'entité que nous examinons est classée comme GPE avant de l'étendre. Le même concept s'applique aux universités, qui devraient être classées comme ORG.

Copiez votre définition de la fonction *enhan_text_with_resource*, étendez-la pour accepter également un type NER comme entrée (ex: *PERSON*, *ORG*, ...) et assurez-vous que l'amélioration du texte ne se produit que si *au moins une entité avec la même forme de surface* du document contient le même type NER que celui fourni au paramètre d'entrée. Cette nouvelle fonction est nommée *enhan_text_with_resource_and_type*.

*NOTE (vous pouvez ignorer - juste pour plus d'informations):* En réalité, nous aimerions que ce soit défini de telle sorte que seule une entité d'un type spécifié ait son ensemble de tokens correspondant dans le texte à lier à la ressource. Cependant, ce processus peut être délicat car la logique impliquera de créer des indicateurs dans le texte pour savoir quelles entités ont déjà été vérifiées (ex: si *Nova Scotia* apparaît deux fois dans le texte, chaque instance avec son propre type NER, alors nous avons besoin pour connaître l'ensemble des tokens que nous éditons pour chacune des entités). Ainsi, vous devez uniquement vous assurer que si au moins une entité de la même forme de surface contient le type NER et se trouve dans le texte, toutes les instances de ces entités sont mises à jour. Si une ressource contient les entités dans le texte, mais qu'elles sont toutes d'un type différent, ignorez-les. Vous êtes libre de mettre en œuvre la méthode la plus robuste détaillée ci-dessus si vous le souhaitez, mais il est recommandé d'appliquer l'approche la plus simpliste que demande la question.

In [109]:
def enhance_text_with_resource_and_type(document_text, resource_df, filename, NER_type):
    '''
    With a NER type, a resource and document's text, enhance any entity found in the resource by linking the entity to
    the appropriate webpage if at least one surface form contains the specified NER type.
    Write the file to the appropriate filename and return the enhanced text
    '''
    enhanced_text = document_text
    # TODO: Parse the document with spaCy
    document_sp = sp(enhanced_text)
    # TODO: Go through the entities and edit the document's text accordingly
    # Note: Be sure to not duplicate your enhancementes
    
    df_ressource = pd.read_csv(resource_df)
    df_ressource.set_index("Text", inplace=True)
    for token in document_sp.ents:
        for i in df_ressource.index:
            if i in token.text:
                types = find_entity_types(token.text, document_sp.ents) #find entity types
                if NER_type in types: #check if the NER_type is in the entity types before enhancing 
                    enhanced_text = enhanced_text.replace(token.text, ("<a href=\"" + str(df_ressource.loc[i]["URL"]) + "\">" + token.text + "</a>"))
    
    # Write the result as an HTML file
    with open(filename, "w", encoding="utf-8") as f:
        f.write(enhanced_text)
        f.close()
    return enhanced_text

**(TO DO) Q10 - 2 points**  
Refaites les tests effectués en Q7 avec la fonction *enhan_text_with_resource* nouvellement définie. Assurez-vous d'utiliser le type NER approprié en fonction de la ressource utilisée pour le linking autorisée.

In [110]:
# Enhance the text for the document below with the US cities
doc = df["text"][6]
enhance_text_with_resource_and_type(doc, "US_Cities.csv", "US_Cities_NER.html", "GPE")

# Enhance the text for the document below with the Canadian provinces
# File extracted from https://en.wikipedia.org/wiki/Provinces_and_territories_of_Canada
doc = df["text"][53]
enhance_text_with_resource_and_type(doc, "Canada_Provinces.csv", "Canada_Provinces_NER.html", "GPE")

# Enhance the text for the document below with the Canadian universities
# File extracted from https://en.wikipedia.org/wiki/List_of_universities_in_Canada
doc = df["text"][53]
enhance_text_with_resource_and_type(doc, "Canada_Universities.csv", "Canada_Universities_NER.html", "ORG")

'The latest:  Federal government announces new measures to mobilize scientists. Saskatchewan unveils five-stage plan to reopen\xa0its\xa0economy. The Calgary Stampede is cancelled for the first time in nearly a century. North Korea claims it has had no coronavirus cases. COVID-19 has brought research into other medical conditions to a grinding halt. Ending Quebec\'s lockdown could mean freedom for some, but not for all. The pandemic is giving some countries political cover for controversial acts. INTERACTIVE |\xa0See the latest data on coronavirus cases\xa0in Canada. Have a coronavirus question or news tip for CBC News? Email: Covid@cbc.ca  The federal government has announced a $1.1-billion plan\xa0to marshal Canada\'s scientific community in the fight against the coronavirus, as some provinces with relatively fewer cases begin to weigh how they will relax restrictions put in place to slow the spread of\xa0COVID-19. Scientists around the globe are scrambling to come up with tests, tre

***SIGNATURE:***
Mon nom est Dounia Mansouri
J'atteste être l'auteur de cette mission.